In [1]:
from pyspark.sql import functions as F
from delta.tables import DeltaTable

# Load indicators table
df = spark.table("gold_stock_daily_indicators")

# Trend direction
df = df.withColumn("trend_direction",
    F.when(F.col("sma_20") > F.col("sma_50"), "bullish")
     .when(F.col("sma_20") < F.col("sma_50"), "bearish")
     .otherwise("neutral")
)

# Momentum score (simplified)
df = df.withColumn("momentum_score",
    (F.col("rsi_14") + F.col("macd_hist") * 10 + (F.col("ema_12") - F.col("ema_26")) * 5)
)

# Volatility regime
df = df.withColumn("volatility_regime",
    F.when(F.col("volatility_30") < 0.01, "low")
     .when(F.col("volatility_30") < 0.03, "medium")
     .otherwise("high")
)

# Breakout signal
df = df.withColumn("breakout_signal",
    F.col("close") > F.col("rolling_high_20")
)

# Mean reversion signal
df = df.withColumn("mean_reversion_signal",
    F.col("close") < F.col("bb_lower")
)

# Trend strength
df = df.withColumn("trend_strength",
    F.abs(F.col("macd")) + F.abs(F.col("ema_12") - F.col("ema_26"))
)

# Add ingestion timestamp
df = df.withColumn("ingestion_time", F.current_timestamp())

# Final columns
final_cols = [
    "ticker", "date",
    "trend_direction", "momentum_score", "volatility_regime",
    "breakout_signal", "mean_reversion_signal", "trend_strength",
    "ingestion_time"
]

df = df.select(*final_cols)

# Save or merge
gold_table = "gold_stock_trend_signals"

if spark.catalog.tableExists(gold_table):
    delta_table = DeltaTable.forName(spark, gold_table)
    (
        delta_table.alias("target")
        .merge(
            df.alias("source"),
            "target.ticker = source.ticker AND target.date = source.date"
        )
        .whenMatchedUpdateAll()
        .whenNotMatchedInsertAll()
        .execute()
    )
else:
    df.write.format("delta").saveAsTable(gold_table)


StatementMeta(, 99043e3d-a250-4666-a13b-3280ab565526, 3, Finished, Available, Finished)